# Pruebas declaraciones

In [1]:
import pandas as pd
import configparser
import openai
from pydantic import BaseModel
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm
import json

In [2]:
data = pd.read_csv("../data/contenido_2023_04_05_matriz_not_julia.csv")
data

,Unnamed: 0,N,n_periodico,enlace,Medio,año,Fecha,Nombreperiodista,Nombreperiodista_A,Género_periodista,...,@25ge3ero1oto,@25perspectiva,@25piedefoto,@26ge3ero1oto,@26perspectiva,@26piedefoto,@27ge3ero1oto,@27perspectiva,@27piedefoto,contenido
0,0,12,12,https://www.infolibre.es/videolibre/como-lo-ve...,1,2021,10/10/2021,Antonio Contreras,ANTONIO CONTRERAS,0,...,,,,,,,,,,"Antonio Lucas (Madrid, 1975) acaba de publicar..."
1,1,438,276,https://www.lavanguardia.com/participacion/car...,2,2021,28/1/2021,ANDREU YAKUBUV-TREMBACH,ANDREU YAKUBUV-TREMBACH,0,...,,,,,,,,,,Distinguir entre las esferas íntima y privada ...
2,2,1411,414,https://www.elmundo.es/economia/2020/04/08/5e8...,3,2020,8/4/2020,EDUARDO FERNÁNDEZ,EDUARDO FERNÁNDEZ,0,...,,,,,,,,,,Directo. Última hora sobre el coronavirus\n\nL...
3,3,1418,421,https://www.elmundo.es/vida-sana/familia-y-co/...,3,2020,20/3/2020,MAR MUÑIZ,MAR MUÑIZ,1,...,,,,,,,,,,Coronavirus: ¿nos convalidarán Teleco por tele...
4,4,1836,839,https://www.elmundo.es/papel/historias/2019/03...,3,2019,3/3/2019,FERMÍN ELIZARI,FERMÍN ELIZARI,0,...,,,,,,,,,,15.000 personas de todo el mundo revisan las p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4122,4130,4130,1325,https://elpais.com/tecnologia/2017/01/17/actua...,4,2017,18/1/2017,MARYA G. NIETO,MARYA G. NIETO,1,...,,,,,,,,,,Dos algoritmos clave en el desarrollo de los a...
4123,4131,4131,1326,https://elpais.com/elpais/2017/01/05/eps/14835...,4,2017,5/1/2017,JAVIER MARTÍN DEL BARRIO,JAVIER MARTÍN DEL BARRIO,0,...,,,,,,,,,,RANA EL KALIOUBY no es una estrella de la canc...
4124,4132,4132,1327,https://elpais.com/elpais/2017/01/18/eps/14846...,4,2017,18/1/2017,GUILLERMO ABRIL,GUILLERMO ABRIL,0,...,,,,,,,,,,"CUANDO ATERRIZÓ en España, en septiembre de 20..."
4125,4133,4134,1329,https://elpais.com/elpais/2017/01/15/opinion/1...,4,2017,16/1/2017,JOAQUÍN PRIETO,JOAQUÍN PRIETO,0,...,,,,,,,,,,Las últimas campañas electorales celebradas en...


In [3]:
data = data[:25]
data

,Unnamed: 0,N,n_periodico,enlace,Medio,año,Fecha,Nombreperiodista,Nombreperiodista_A,Género_periodista,...,@25ge3ero1oto,@25perspectiva,@25piedefoto,@26ge3ero1oto,@26perspectiva,@26piedefoto,@27ge3ero1oto,@27perspectiva,@27piedefoto,contenido
0,0,12,12,https://www.infolibre.es/videolibre/como-lo-ve...,1,2021,10/10/2021,Antonio Contreras,ANTONIO CONTRERAS,0,...,,,,,,,,,,"Antonio Lucas (Madrid, 1975) acaba de publicar..."
1,1,438,276,https://www.lavanguardia.com/participacion/car...,2,2021,28/1/2021,ANDREU YAKUBUV-TREMBACH,ANDREU YAKUBUV-TREMBACH,0,...,,,,,,,,,,Distinguir entre las esferas íntima y privada ...
2,2,1411,414,https://www.elmundo.es/economia/2020/04/08/5e8...,3,2020,8/4/2020,EDUARDO FERNÁNDEZ,EDUARDO FERNÁNDEZ,0,...,,,,,,,,,,Directo. Última hora sobre el coronavirus\n\nL...
3,3,1418,421,https://www.elmundo.es/vida-sana/familia-y-co/...,3,2020,20/3/2020,MAR MUÑIZ,MAR MUÑIZ,1,...,,,,,,,,,,Coronavirus: ¿nos convalidarán Teleco por tele...
4,4,1836,839,https://www.elmundo.es/papel/historias/2019/03...,3,2019,3/3/2019,FERMÍN ELIZARI,FERMÍN ELIZARI,0,...,,,,,,,,,,15.000 personas de todo el mundo revisan las p...
5,5,2001,1004,https://www.elmundo.es/cronica/2018/10/27/5bcc...,3,2018,27/10/2018,LUIS BLÁZQUEZ,LUIS BLÁZQUEZ,0,...,,,,,,,,,,En el Teatro Ateneo me espera Sophia. Pese a s...
6,6,3447,642,https://elpais.com/tecnologia/2018/04/06/actua...,4,2018,8/4/2018,SONIA CORONA,SONIA CORONA,1,...,,,,,,,,,,Sophia fue creada en 2016 por el estadounidens...
7,7,3778,973,https://elpais.com/tecnologia/2017/09/05/actua...,4,2017,8/9/2017,DANIEL POLANI THE CONVERSATION,DANIEL POLANI THE CONVERSATION,0,...,,,,,,,,,,En los tiempos que corren es cada vez más difí...
8,8,87,87,https://www.infolibre.es/medios/marta-garcia-a...,1,2020,15/6/2020,Miriam Puelles,Miriam Puelles,1,...,,,,,,,,,,"Hay situaciones que escapan a nuestro control,..."
9,9,129,129,https://www.infolibre.es/cultura/esther-vivas-...,1,2019,20/3/2019,Clara Morales,Clara Morales,1,...,,,,,,,,,,"Dice Esther Vivas (Sabadell, 1975) que ""la lit..."


In [4]:
class Quote(BaseModel):
    cita_textual: str
    fuente: str
    genero_fuente: str
    colectivos: list[str]
    verbo: str
    tipo_fuente: int

class ResponseFormat(BaseModel):
    quotes: list[Quote]
    num_quotes: int

config = configparser.ConfigParser()
config.read('config.ini')

OPENAI_MODEL = config['DEFAULT']['OPENAI_MODEL']
openai.api_key = config['DEFAULT']['OPENAI_API_KEY']

In [5]:
predictions = []

# Loop de predicción
for idx, row in tqdm(data.iterrows(), total=len(data), desc="Procesando predicciones"):
    n = row['N']
    n_periodico = row['n_periodico']
    contenido = row['contenido']
    try:
        # Llamada al modelo para detectar las citas humanas y sus fuentes
        completion = openai.beta.chat.completions.parse(
            model=OPENAI_MODEL,
            messages=[
                {
                    "role": "system", 
                    "content": "Eres un periodista especializado en la extracción de declaraciones de humanos en artículos de noticias. Tu tarea es identificar y extraer todas las declaraciones humanas del texto, incluyendo la cita textual (si está entre comillas), la fuente de la declaración (como nombre, cargo o afiliación) y el verbo o expresión utilizada para introducir la declaración. Debes ignorar cualquier declaración hecha por robots o entidades no humanas."
                },
                {
                    "role": "user", 
                    "content": f"""
                                Extrae todas las declaraciones humanas del siguiente artículo, centrándote en identificar las personas mencionadas y su género. Para cada declaración, proporciona:

                                - **Cita textual**: Si está presente en el texto.
                                - **Fuente**: La persona, cargo o afiliación que hizo la declaración.
                                - **Género de la fuente**: El género correspondiente de la persona que hizo la declaración (masculino, femenino, desconocido).
                                - **Verbo**: El verbo o expresión utilizada para introducir la declaración.
                                - **Colectivos en la declaración**: Grupos sociales a los que hace referencia la declaración. Identifica tanto colectivos explícitos como aquellos implícitos en el contexto. 
                                - **Tipo de fuente**: Clasificación de la fuente según la siguiente lista:
                                
                                0 = Director/a o presidente/a  
                                1 = Investigador/a  
                                2 = Alto Cargo Directivo/a  
                                3 = Político/a  
                                4 = Trabajador/a  
                                5 = Experto/a  
                                6 = Periodista  
                                7 = Escritor/a  
                                8 = Médico/a  
                                9 = Economista  
                                10 = Ciudadano/a  
                                11 = Abogado/a  
                                12 = Músico/a  
                                13 = Actor/Atriz  
                                14 = Alumno/a  
                                15 = Deportista  
                                16 = Analista  
                                17 = Artista  
                                18 = Famoso/a  
                                19 = Arquitecto/a  
                                20 = Dir. Cine / guionista  
                                21 = Activista  
                                22 = Rey/Reina  
                                23 = Personaje de ficción  
                                24 = El Papa
                                100 = Desconocido  

                                Si el género no es explícito en el texto, márcalo como "desconocido".  

                                Ignora declaraciones hechas por robots o entidades no humanas.

                                Texto: {contenido}
                                """
                                                }
            ],
            response_format=ResponseFormat
        )
        
        # Extraer la respuesta
        response_message = completion.choices[0].message.content
        print(response_message)

        # Usamos json.loads para convertir el string JSON a un objeto Python
        response_data = json.loads(response_message)

        # Agregar el índice de la noticia al JSON
        response_data['indice'] = idx  # Agrega el índice al diccionario de cada cita
        response_data['N'] = n
        response_data['n_periodico'] = n_periodico
        
        # Agregar el resultado con el índice a las predicciones
        predictions.append(response_data)

    except Exception as e:
        print(f"Error en el índice {idx}: {e}")

Procesando predicciones:   4%|▍         | 1/25 [00:52<21:01, 52.57s/it]

{"quotes":[{"cita_textual":"\"Yo encontré en esos hombres el quilate más puro del humano: la nobleza, la lealtad, la bondad, una bondad sin orillas. La lección que yo me llevo de Gran Sol, curiosamente, no es el mar. Son los hombres que surcan un mar, que saben que su vida es aquello que ese mismo mar no les ha querido arrancar\"","fuente":"Antonio Lucas","genero_fuente":"masculino","colectivos":["marineros","hombres"],"verbo":"encontré","tipo_fuente":7},{"cita_textual":"\"Al trabajar en un periódico, aunque yo siempre he pertenecido a la parte de cultura, te impregnas del día a día, de ese pulso, de esa vibración de la política. Participé un par de años en alguna tertulia política en radio y me di cuenta de que no era mi territorio porque yo sé hablar bien de las cosas que me importan o de las cosas que me enfadan, pero me cuesta muchísimo ponerme a hablar de las cosas que otros propician o generan o hacen para que a nosotros nos gusten o nos enfaden, en este caso, los políticos. Yo t

Procesando predicciones:   8%|▊         | 2/25 [01:01<10:19, 26.91s/it]

{"quotes":[{"cita_textual":"Aquélla es más amplia que ésta (…) constituye un conjunto más global de facetas de la personalidad que, aisladamente consideradas, pueden carecer de significación intrínseca, pero que, coherentemente enlazadas entre sí, arrojan como precipitado un retrato de la personalidad del individuo que éste tiene derecho a mantener reservado.","fuente":"No se especifica","genero_fuente":"desconocido","colectivos":["ciudadanos"],"verbo":"expres","tipo_fuente":100},{"cita_textual":"el ‘no hay derecho’ que vivimos es otro.","fuente":"No se especifica","genero_fuente":"desconocido","colectivos":[],"verbo":"afirmar","tipo_fuente":100},{"cita_textual":"advierte voces tan autorizadas en la materia como Craig y Ludloff.","fuente":"Craig y Ludloff","genero_fuente":"desconocido","colectivos":[],"verbo":"advertir","tipo_fuente":100},{"cita_textual":"la puesta en valor de la privacidad asienta lo que viene a ser nuestro modo de vida europeo.","fuente":"No se especifica","genero_fu

Procesando predicciones:  12%|█▏        | 3/25 [01:03<05:40, 15.46s/it]

...

{"quotes":[{"cita_textual":"\"Es imposible\"","fuente":"La Autoridad Independiente de Responsabilidad Fiscal (AIReF)","genero_fuente":"desconocido","colectivos":["Gobierno","ciudadanos"],"verbo":"afirma","tipo_fuente":10},{"cita_textual":"\"Más allá de las líneas estratégicas descritas en el documento 'España Puede', no se conoce ni el detalle de los proyectos aprobados, ni los plazos de ejecución, ni el contenido de las reformas estructurales que acompañarán al Plan en áreas nucleares para el crecimiento económico como el mercado de trabajo, el sistema de pensiones o la reforma tributaria\"","fuente":"Cristina Herrero, presidenta de la AIReF","genero_fuente":"femenino","colectivos":["Gobierno","ciudadanos"],"verbo":"critica","tipo_fuente":0},{"cita_textual":"\"Sin esta información es imposible juzgar los efectos multiplicadores sobre el crecimiento y el empleo, la capacidad de atracción de la inversión privada, ni sus potenciales efectos sobre el potencial de crecimiento de la economí

Procesando predicciones:  96%|█████████▌| 24/25 [05:08<00:13, 13.63s/it]

{"quotes":[{"cita_textual":"La prostitución sigue funcionando, porque los clientes encuentran triquiñuelas para ir a los pisos","fuente":"Mabel Lozano","genero_fuente":"femenino","colectivos":["prostitutas","clientes de prostitución"],"verbo":"dijo","tipo_fuente":7},{"cita_textual":"Ha sido terrible. Las prostitutas son las más vulnerables: algunas han llegado a pasar mucha hambre, otras han aumentado su deuda y no han tenido ni 10 dólares para mandar a sus países de origen... y en numerosos casos hemos visto que la prostitución ha migrado a las fronteras digitales.","fuente":"Mabel Lozano","genero_fuente":"femenino","colectivos":["prostitutas","mujeres migrantes"],"verbo":"dijo","tipo_fuente":7},{"cita_textual":"Absolutamente a favor de que introduzca el tema de las tercerías locativas. Pero podría haber sido mucho más valiente y haber abordado la fiscalía especializada.","fuente":"Mabel Lozano","genero_fuente":"femenino","colectivos":["mujeres","victimas de prostitución"],"verbo":"di

Procesando predicciones: 100%|██████████| 25/25 [05:26<00:00, 13.05s/it]

{"quotes":[{"cita_textual":"\"Desde foros especializados y también desde los medios de comunicación no hacemos más que dar vueltas una y otra vez al tema de la falta de vocaciones STEAM en las niñas y jóvenes.\"","fuente":"Meritxell Bautista","genero_fuente":"femenino","colectivos":["niñas","jóvenes"],"verbo":"habla de","tipo_fuente":0},{"cita_textual":"\"Los estudios dicen que una niña descarta profesiones vinculadas a la ciencia y tecnología a partir de los ¡6 años!!!\"","fuente":"Meritxell Bautista","genero_fuente":"femenino","colectivos":["niñas"],"verbo":"dice que","tipo_fuente":0},{"cita_textual":"\"Quiero ayudar a una adolescente a ver por qué la tecnología es sexy y divertida.\"","fuente":"Meritxell Bautista","genero_fuente":"femenino","colectivos":["adolescentes"],"verbo":"me gustaría decirle a","tipo_fuente":0},{"cita_textual":"\"La sociedad necesita menos #influencers y más #Sciencefluencers\"","fuente":"Ana Freire","genero_fuente":"desconocido","colectivos":[],"verbo":"dice

In [6]:
# Guardar las predicciones en un archivo JSON
with open('predictions_declaraciones.json', 'w', encoding='utf-8') as f:
    json.dump(predictions, f, ensure_ascii=False, indent=4)

In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error


# Extraer valores numéricos
y_pred = [p["num_quotes"] for p in predictions]
y_true = data["ndeclaracion"].tolist()

# Calcular errores
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = mse ** 0.5  # Raíz del error cuadrático medio

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")


MAE: 2.96
MSE: 15.52
RMSE: 3.94


In [22]:
print(f"{'Pred. NDeclaracion':<25}{'NDeclaracion':<15}{'+/-'}")
print("=" * 50)

for pred, ndecl in zip(predictions, data["ndeclaracion"]):
    diff = pred["num_quotes"] - ndecl
    print(f"{pred['num_quotes']:<25}{ndecl:<15}{diff:+}")


Pred. NDeclaracion       NDeclaracion   +/-
10                       0              +10
5                        0              +5
1                        0              +1
7                        0              +7
2                        0              +2
4                        1              +3
3                        0              +3
5                        0              +5
10                       15             -5
14                       15             -1
6                        6              +0
11                       11             +0
6                        9              -3
8                        7              +1
10                       18             -8
5                        10             -5
11                       12             -1
4                        7              -3
5                        5              +0
15                       11             +4
10                       10             +0
5                        6              -1
8        

In [7]:
def compute_metrics(true_labels, predicted_labels):

    # Obtener reporte completo
    report = classification_report(true_labels, predicted_labels, output_dict=True)
    
    # Obtener la matriz de confusión
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    
    # Extraer métricas para cada clase y globales
    metrics = {
        'accuracy': report['accuracy'],
        'weighted_f1': report['weighted avg']['f1-score'],
        'macro_f1': report['macro avg']['f1-score'],
        # 'weighted_precision': report['weighted avg']['precision'],
        # 'weighted_recall': report['weighted avg']['recall'],
        # La matriz de confusión no se incluye normalmente como una métrica devuelta porque no es un escalar
        'confusion_matrix': conf_matrix.tolist()  # Convertir a lista para asegurarse de que es serializable si es necesario
    }
    
    # # Añadir métricas específicas por clase si se requiere
    # for label, scores in report.items():
    #     if label not in ["accuracy", "macro avg", "weighted avg"]:
    #         metrics[f'{label}_precision'] = scores['precision']
    #         metrics[f'{label}_recall'] = scores['recall']
    #         metrics[f'{label}_f1'] = scores['f1-score']
    #         metrics[f'{label}_support'] = scores['support']
    
    return metrics

In [8]:
# true_labels = data['n_declaraciones']
# predicted_labels = train_df['predicted_emotion']
# print(len(true_labels))
# print(len(predicted_labels))

# metrics = compute_metrics(true_labels, predicted_labels)
# metrics